First off, just read the data

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("results_with_data.csv")

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
df

,App,Key,Logical Lines of Code,Halstead Volume,Halstead Difficulty,Halstead Effort,Maintainability Index,Cyclomatic Complexity,Cognitive Complexity,Number of Words
0,insurance_claims,fbp_app_min,210,543.658677,6.210526,3376.406518,43.30,1.934783,0.424242,846
1,insurance_claims,fbp_app_data,239,610.424232,6.190476,3778.816672,40.80,2.125000,0.657143,974
2,insurance_claims,fbp_app_ml,263,543.658677,6.848214,3723.091116,41.85,2.294118,0.918919,1124
3,insurance_claims,soa_app_min,250,383.285425,6.000000,2299.712549,46.22,1.900000,1.166670,1094
4,insurance_claims,soa_app_data,285,463.722322,7.000000,3246.056251,44.19,1.941176,1.176470,1247
5,insurance_claims,soa_app_ml,356,675.754180,7.101449,4798.834030,38.49,2.102564,1.487180,1508
6,mblogger,fbp_app_min,172,155.114511,2.000000,310.229021,44.10,1.810811,0.307692,529
7,mblogger,fbp_app_data,275,372.494509,3.000000,1117.483526,31.51,1.931034,0.525000,830
8,mblogger,fbp_app_ml,340,560.046581,4.066667,2277.522764,26.50,1.970149,0.851064,1055
9,mblogger,soa_app_min,240,213.673439,3.000000,641.020318,46.93,1.821429,1.142860,1002


Some metrics scale with the size of the codebase, while others don't. For the former we calculate change in percentages, for the latter in absolute values.

In [7]:
absolute_diff_metrics = ["Halstead Difficulty", "Maintainability Index", "Cyclomatic Complexity", "Cognitive Complexity"]
percentage_diff_metrics = ["Logical Lines of Code", "Halstead Volume", "Halstead Effort", "Number of Words"]

Here we calculate the differences and re-organize the dataframe a bit.

In [8]:
all_dfs = []

for i in range(0, df.shape[0], 3):
    df_change = pd.concat([
        df.iloc[i:i+3, 2:][percentage_diff_metrics].pct_change(),
        df.iloc[i:i+3, 2:][absolute_diff_metrics].diff(),
    ], axis=1)
    df_change = df_change.iloc[1: , :]
    df_change.insert(0, "Keys", ["min/data", "data/ml"])
    all_dfs.append(df_change)

fbp_dfs = [all_dfs[0], all_dfs[2], all_dfs[4]]
soa_dfs = [all_dfs[1], all_dfs[3], all_dfs[5]]
    
for fbp_df in fbp_dfs:
    fbp_df.insert(0, "Paradigm", ["fbp", "fbp"])
for soa_df in soa_dfs:
    soa_df.insert(0, "Paradigm", ["soa", "soa"])

all_changes_df = pd.concat(fbp_dfs + soa_dfs, axis=0)
all_changes_df

,Paradigm,Keys,Logical Lines of Code,Halstead Volume,Halstead Effort,Number of Words,Halstead Difficulty,Maintainability Index,Cyclomatic Complexity,Cognitive Complexity
1,fbp,min/data,0.138095,0.122808,0.119183,0.151300,-0.020050,-2.50,0.190217,0.232901
2,fbp,data/ml,0.100418,-0.109376,-0.014747,0.154004,0.657738,1.05,0.169118,0.261776
7,fbp,min/data,0.598837,1.401416,2.602124,0.568998,1.000000,-12.59,0.120224,0.217308
8,fbp,data/ml,0.236364,0.503503,1.038082,0.271084,1.066667,-5.01,0.039115,0.326064
13,fbp,min/data,0.171533,0.223463,0.451884,0.207113,1.094453,-4.47,0.272904,0.653398
14,fbp,data/ml,0.121495,0.148057,0.157376,0.126516,0.056465,-2.71,0.023392,0.009760
4,soa,min/data,0.140000,0.209862,0.411505,0.139854,1.000000,-2.03,0.041176,0.009800
5,soa,data/ml,0.249123,0.457239,0.478358,0.209302,0.101449,-5.70,0.161388,0.310710
10,soa,min/data,0.104167,0.000000,0.000000,0.135729,0.000000,-1.82,0.078571,-0.009530
11,soa,data/ml,0.358491,0.921046,2.344043,0.352373,2.222222,-7.84,0.152632,0.235090


In the resulting dataframe columns contain measured changes for different apps. "Keys" column signifies which pair is considered. E.g. "min/data" means that changes are measured for transition between "min" and "data" stages.

Let's look at individual stages, or rather pairs of stages.

In [9]:
min_data_changes_df = all_changes_df[all_changes_df["Keys"] == 'min/data']
data_ml_changes_df = all_changes_df[all_changes_df["Keys"] == 'data/ml']

In [10]:
min_data_changes_df

,Paradigm,Keys,Logical Lines of Code,Halstead Volume,Halstead Effort,Number of Words,Halstead Difficulty,Maintainability Index,Cyclomatic Complexity,Cognitive Complexity
1,fbp,min/data,0.138095,0.122808,0.119183,0.151300,-0.020050,-2.50,0.190217,0.232901
7,fbp,min/data,0.598837,1.401416,2.602124,0.568998,1.000000,-12.59,0.120224,0.217308
13,fbp,min/data,0.171533,0.223463,0.451884,0.207113,1.094453,-4.47,0.272904,0.653398
4,soa,min/data,0.140000,0.209862,0.411505,0.139854,1.000000,-2.03,0.041176,0.009800
10,soa,min/data,0.104167,0.000000,0.000000,0.135729,0.000000,-1.82,0.078571,-0.009530
16,soa,min/data,0.215247,0.158409,0.277145,0.204927,0.492901,-3.87,0.010101,0.026260


In [11]:
data_ml_changes_df

,Paradigm,Keys,Logical Lines of Code,Halstead Volume,Halstead Effort,Number of Words,Halstead Difficulty,Maintainability Index,Cyclomatic Complexity,Cognitive Complexity
2,fbp,data/ml,0.100418,-0.109376,-0.014747,0.154004,0.657738,1.05,0.169118,0.261776
8,fbp,data/ml,0.236364,0.503503,1.038082,0.271084,1.066667,-5.01,0.039115,0.326064
14,fbp,data/ml,0.121495,0.148057,0.157376,0.126516,0.056465,-2.71,0.023392,0.009760
5,soa,data/ml,0.249123,0.457239,0.478358,0.209302,0.101449,-5.70,0.161388,0.310710
11,soa,data/ml,0.358491,0.921046,2.344043,0.352373,2.222222,-7.84,0.152632,0.235090
17,soa,data/ml,0.099631,0.098153,0.212133,0.081340,0.550276,-2.45,0.047149,0.113100
